In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# pip install git+https://github.com/qubvel/segmentation_models
# pip install git+https://github.com/qubvel/segmentation_models
!pip install efficientnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 8.1 MB/s eta 0:00:00


In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Add, Multiply, Reshape, Lambda
from tensorflow.keras import backend as K
from efficientnet.tfkeras import EfficientNetB3
# from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import AdamW

In [8]:
# Load data
x_train = np.load('/content/drive/MyDrive/np_images/x_train.npy')
y_train = np.load('/content/drive/MyDrive/np_images/y_train.npy')
x_test = np.load('/content/drive/MyDrive/np_images/x_test.npy')
y_test = np.load('/content/drive/MyDrive/np_images/y_test.npy')

In [29]:
# def preprocess_image(img):

#     img=np.load(img)
#     # Crop the center 80% of the image
#     height, width, channels = img.shape
#     crop_height = int(height * 0.8)
#     crop_width = int(width * 0.8)
#     start_height = int((height - crop_height) / 2)
#     start_width = int((width - crop_width) / 2)
#     img = img[start_height:start_height+crop_height, start_width:start_width+crop_width, :]
    
#     # Apply Gaussian blur with kernel size 3
#     img = cv2.GaussianBlur(img, (3, 3), 0)
    
#     # Rescale the pixel values to [0, 1]
    
#     return img

In [30]:
# import os
# from matplotlib import pyplot as plt
# import cv2 # for image pre-processing
# import gc
# from tqdm import tqdm

# #preprocessed_x_train = np.empty((len(x_train), 256, 256, 3), dtype=np.float32)

# for i in range(len(x_train)):
#     img = x_train[i]
#     img = preprocess_image(img)
#     img = cv2.resize(img, (224, 224))
#     x_train[i] = img

# # preprocessed_x_test = np.empty((len(x_test), 256, 256, 3), dtype=np.float32)

# for i in range(len(x_test)):
#     img = x_test[i]
#     img = preprocess_image(img)
#     img = cv2.resize(img, (224, 224))
#     x_test[i] = img

In [9]:
# resize
x_train = tf.image.resize(x_train, size=(224, 224)) / 255.0
x_test = tf.image.resize(x_test, size=(224, 224)) / 255.0

In [10]:
import cv2

In [11]:
def preprocess_image(img):
    # Crop the center 80% of the image
    #print(img.shape)
    height, width, channels = img.shape
    crop_height = int(height * 0.8)
    crop_width = int(width * 0.8)
    start_height = int((height - crop_height) / 2)
    start_width = int((width - crop_width) / 2)
    img = img[start_height:start_height+crop_height, start_width:start_width+crop_width, :]
    
    # Apply Gaussian blur with kernel size 3
    img = cv2.GaussianBlur(img, (3, 3), 0)

    img = cv2.resize(img, (224, 224))
    
    # Rescale the pixel values to [0, 1]
    img = img / 255.
    
    # Rescale the pixel values to [0, 1]
    
    return img

In [14]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
def create_datagen():
    return ImageDataGenerator(
        horizontal_flip=True,   # randomly flip images
        vertical_flip=True,     # randomly flip images

        preprocessing_function = preprocess_image
    )
    
# Using original generator
data_generator = create_datagen().flow(x_train, y_train,batch_size=32)
print("Image data augmentated ...")

def spatial_attention(input_tensor):
    x = Conv2D(1, kernel_size=7, strides=1, padding='same', activation='sigmoid')(input_tensor)
    return Multiply()([input_tensor, x])

def channel_attention(input_tensor):
    x = GlobalAveragePooling2D()(input_tensor)
    x = Dense(K.int_shape(input_tensor)[-1] // 4, activation='relu')(x)
    x = Dense(K.int_shape(input_tensor)[-1], activation='sigmoid')(x)
    x = Reshape((1, 1, K.int_shape(input_tensor)[-1]))(x)
    return Multiply()([input_tensor, x])

def efficientnet_sa(num_classes):
    base_model = EfficientNetB3(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

    x = base_model.output
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = spatial_attention(x)
    x = channel_attention(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = keras.models.Model(inputs=base_model.input, outputs=output)

    for layer in base_model.layers:
        layer.trainable = False

    return model




# Split data into train and validation sets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(32)

# f1 score
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior() 
import keras.backend as K

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

# ---------------------------------------------------------------------
# Create model
model = efficientnet_sa(num_classes=5)
model.compile(loss='categorical_crossentropy', optimizer=AdamW(lr=0.0001), metrics=['accuracy',f1])

# Train model
history = model.fit(train_dataset, epochs=100, validation_data=test_dataset)

# Evaluate model on test set
# test_loss, test_acc = model.evaluate(test_dataset)
# print('Test loss:', test_loss)
# print('Test accuracy:', test_acc)


Image data augmentated ...


Epoch 1/100
92/92 [==============================] - 30s 198ms/step - loss: 0.9305 - accuracy: 0.6603 - f1: 0.2180 - val_loss: 0.7913 - val_accuracy: 0.7203 - val_f1: 0.2369
Epoch 2/100
92/92 [==============================] - 13s 141ms/step - loss: 0.7824 - accuracy: 0.7139 - f1: 0.2875 - val_loss: 0.7371 - val_accuracy: 0.7394 - val_f1: 0.2174
Epoch 3/100
92/92 [==============================] - 13s 141ms/step - loss: 0.7414 - accuracy: 0.7228 - f1: 0.3061 - val_loss: 0.7184 - val_accuracy: 0.7353 - val_f1: 0.3357
Epoch 4/100
92/92 [==============================] - 13s 146ms/step - loss: 0.7165 - accuracy: 0.7310 - f1: 0.3237 - val_loss: 0.7017 - val_accuracy: 0.7449 - val_f1: 0.2997
Epoch 5/100
92/92 [==============================] - 13s 143ms/step - loss: 0.6955 - accuracy: 0.7269 - f1: 0.3265 - val_loss: 0.6824 - val_accuracy: 0.7476 - val_f1: 0.3688
Epoch 6/100
92/92 [==============================] - 13s 142ms/step - loss: 0.6811 - accuracy: 0.7412 - f1: 0.3437 - val_loss: 0.6

In [15]:

# convert the history.history dict to a pandas DataFrame:     
import pandas as pd
hist_df = pd.DataFrame(history.history) 

#  saving history to csv file
hist_csv_file = '/content/drive/MyDrive/efficientnet_attention_model/efficientnet_attention_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

# saving the model
model.save('/content/drive/MyDrive/efficientnet_attention_model/efficientnet_attention.h5')
